In [1]:
import nltk
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Load the Data and Looking at the Header

In [2]:
Data = pd.read_csv("Data.csv", encoding = "ISO-8859-1")
Data.head(5)

C:\Users\ali\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (25,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SR Number,Incident Date,Reported Date,SEVERITY,Last Modified,SR Type,SR Status,Plant Number,SR Group,SR Owner,...,SR Summary,Comments,Actual Labor Hours,VALID_ALTEC_UNIT_SERIAL,Unit Age at Incident Date,SR_NOTE_CALL_REASON,Days to Work Complete,SR_NOTE_CAUSE,SR_NOTE_CORRECTION,CUST_ACCOUNT_ID
0,1690559,8/3/2011,8/3/2011,Partially Operable,9/28/2011,Plant Repair Order,Closed,333,0333 COLORADO SERVICE CENTER,"0333 Martin, Annie",...,COMPLETE FOOT PEDAL WIRING,Entering charges from dealer for programming,28.37,0899BA1913,12.01,COMPLETE FOOT PEDAL WIRING,28,067-0001854 PER CUSTOMER REQUEST,1,183949
1,2108988,5/16/2014,5/16/2014,Partially Operable,7/15/2014,Plant Repair Order,Closed,2445,LTD SVC DISPATCHERS,"2445 Ireland, Mark",...,"inspection, dielectric. leak at digger(fitting...",-,39.83,1108DV3955,5.54,-- REPAIR REAR BUMPER ASSEMBLY\n-- TOW EYES BE...,-,-,-- CUT OFF AND REPLACED DAMAGED AREA OF REAR B...,15941
2,1671881,5/12/2011,5/12/2011,Fully Operational,5/31/2011,Road Repair Order,Closed,111,SMW2 PLANNER GROUP,"Faulks, Bodie A",...,"ANNUAL PM, DIELECTRIC AND DOT INSPECTION",-,5.00,0307DV2684,4.20,DOT INSPECTION DIELECTRIC TEST PM INSPECTION,5,ANNUAL DUE,--CSNS HAVENT BEEN DONE\nDO DOT INSPECTION.\nD...,15204
3,1645303,1/28/2011,1/28/2011,Partially Operable,3/29/2011,Plant Repair Order,Closed,2443,LTD SVC DISPATCHERS,"2443 Kerr, Brian J",...,SERVICE REQUEST,-,18.25,0908DV3833,2.41,- GLOBAL RENTAL RETURN,-,- JR 672,- 672\n 8-Feb-11\n\n DM47TR\n 160-118386...,15941
4,1679223,6/14/2011,6/14/2011,Partially Operable,6/17/2011,Plant Repair Order,Closed,2442,LTD SVC DISPATCHERS,"2442 Elias, Kerry",...,SERVICE REQUEST,-,2.00,1096BE1775,14.71,- RH PLATFORM OUT OF ADJUSTMENT\n - HYDRAULIC ...,2,- UNKNOWN\n,- ADJUST RH & LH PLATFORMS\n - REMOVE AND REPL...,27292


###  Column names of the original data

In [3]:
Data.columns

Index(['SR Number', 'Incident Date', 'Reported Date', 'SEVERITY',
       'Last Modified', 'SR Type', 'SR Status', 'Plant Number', 'SR Group',
       'SR Owner', 'Customer Phone', 'Contact Name', 'Contact Email',
       'Contact Phone', 'Ship To Address', 'Ship To Address.1', 'Ship To City',
       'Ship To State', 'Ship To Zip', 'Bill To Address', 'Bill To Address.1',
       'Bill To City', 'Bill To State', 'Bill To Zip', 'Instance', 'Owner ID',
       'Instance Description', 'Model', 'Serial #',
       'sum(ACTUAL_LABOR_HOURS)\n', 'Unit Serial Number', 'Tag',
       'In-Service Date', 'SR Summary', 'Comments', 'Actual Labor Hours',
       'VALID_ALTEC_UNIT_SERIAL', 'Unit Age at Incident Date',
       'SR_NOTE_CALL_REASON', 'Days to Work Complete', 'SR_NOTE_CAUSE',
       'SR_NOTE_CORRECTION', 'CUST_ACCOUNT_ID'],
      dtype='object')

### Dimension of the original data (Shape)

In [4]:
Data.shape

(49010, 43)

### Columns of Needed Data

We throw away other columns of this dataset and just focus on "review" columns an dthe "label" column. 

In [5]:
selected =  Data.loc[:, ["SR Summary", "Actual Labor Hours","Days to Work Complete"]] 

### Examples of reviews and the corresponding label 

Labels are given at the end of the reviews

In [6]:
for item in np.array(selected.head(2)):
    print(item)

['COMPLETE FOOT PEDAL WIRING' 28.37 '28']
['inspection, dielectric. leak at digger(fitting in cab) ,remove extension shaft from winch,emg. stop not working at controls, boom cuts out(wait or play with outriggers works)'
 39.83 '-']


### We use pd.loc to extract reviews as a list


In [7]:
raw_Summary = selected.loc[:, ["SR Summary"]] ## Gives the list of all summary 
raw_Summary = raw_Summary["SR Summary"]
raw_Summary[2] ## The fifth summary in the dataset

'ANNUAL PM, DIELECTRIC AND DOT INSPECTION'

### Exctracting Non-Alphanumeric and other Weird Characters!


In [66]:
weird_chars = []
for char in raw_Summary:
    weird_chars += re.findall(r"[^a-zA-Z0-9\s\w\.]", char.lower())
weird_char_list = list(set(weird_chars))
print(weird_char_list)

[',', '`', '*', '?', '&', '#', '>', '/', '%', '-', '\\', "'", ';', '!', '@', '[', '"', ')', '(', ':', '=', '$']


In [67]:
"".join(weird_char_list)

',`*?&#>/%-\\\';!@[")(:=$'

### Lowercasing, removing punctuations, and weird characters


In [68]:
Clean_Summary = [re.sub(r"[Î¾î\'\\\%!\$\&\'()\*\+,\-\.\/:;<=>?@\[\]^_`{|}~]", "", item).lower() for item in raw_Summary]
Clean_Summary[2] ## This does not have anymore any weird char

'annual pm dielectric and dot inspection'

## Replacing the review columns by the processed & clean reviews

In [69]:
selected[["SR Summary"]] = pd.DataFrame(Clean_Summary)
selected[10:20]

,SR Summary,Actual Labor Hours,Days to Work Complete
10,service request,1.00,8
11,service request,12.50,-
12,service request,2.00,3
13,service request,5.53,56
14,check rear throttle oil leak,19.99,5
15,service request,38.14,20
16,service request,1.00,39
17,service request,8.00,27
18,service request,3.75,17
19,annual inspection,32.89,-


###test1

In [78]:
col_summary =  Data["SR Summary"]
labels = list(set(col_summary)) # Original labels
print("The number of original labels is ", len(labels))
print("The number of original labels is ", len(col_summary))

The number of original labels is  36796
The number of original labels is  49010


### Distribution of the Original (multi-) labels


In [79]:
dict_of_distrb = nltk.FreqDist(col_summary)
sorted_raw_label_req = dict_of_distrb.most_common()
#sorted_raw_label_req = sorted(dict_of_distrb.items(), key = lambda x: x[1], reverse = True)
sorted_raw_label_req


[('UCR', 788),
 ('PM INSPECTION', 453),
 ('PM INSPECTION AND DIELECTRIC TEST', 245),
 ('DIELECTRIC TEST', 201),
 ('PM, DIELECTRIC', 200),
 ('PM', 181),
 ('SERVICE REQUEST', 178),
 ('PERFORM UCR', 169),
 ('HYDRAULIC LEAK', 156),
 ('REPAIRS FROM INSPECTION', 153),
 ('BOOM STUCK IN THE AIR', 134),
 ('NO BOOM FUNCTIONS', 129),
 ('RENTAL RETURN - PREP FOR RENT', 117),
 ('HYD LEAK', 106),
 ('BOOM STUCK IN AIR', 90),
 ('PM AND DIELECTRIC', 89),
 ('VISUAL INSPECTION AND DIELECTRIC TEST', 82),
 ('INSPECTION AND DIELECTRIC TEST', 67),
 ('INSPECTION, ANNUAL PM AND DIELECTRIC TEST', 61),
 ('UNIT STUCK IN THE AIR', 59),
 ('BOOM INOPERABLE', 59),
 ('PM INSPECTION, DIELECTRIC TEST', 59),
 ('DIELECTRIC', 59),
 ('ANNUAL INSPECTION', 55),
 ('CHECK UNIT IN/OUT', 55),
 ('BOOM INOP', 54),
 ('REPAIRS FROM PM INSPECTION', 54),
 ('PTO WILL NOT ENGAGE', 50),
 ('PM AND DIELECTRIC TEST', 50),
 ('UNIT STUCK IN HOP', 47),
 ('SHORT UCR', 47),
 ('PM INSPECTION AND DOT INSPECTION', 46),
 ('BLOWN HOSE', 45),
 ('CAL OS